Import Library

In [1]:
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np  # Tambahkan import numpy

# Unduh stopwords jika belum
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rebecca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Read and Print Data

In [2]:
# Baca file CSV
df = pd.read_csv('data/google-play-rev-gen-2.csv')

print(df.columns)

# Tampilkan isi kolom 'Nama'
df.head(10)

Index(['id', 'title', 'avatar', 'rating', 'snippet', 'likes', 'date',
       'iso_date', 'response'],
      dtype='object')


,id,title,avatar,rating,snippet,likes,date,iso_date,response
0,e5384431-56f9-43fa-a32a-53296afc7f66,Seraphim,https://play-lh.googleusercontent.com/a-/ALV-U...,3.0,"While I've reviewed this before, I decided to ...",88,"October 09, 2024",2024-10-09T00:08:20Z,NaN
1,6a73081f-3490-47ba-89fa-83744cb20940,TWOSTORE !,https://play-lh.googleusercontent.com/a-/ALV-U...,3.0,Very fun but I wish there was more fighting in...,35,"October 12, 2024",2024-10-12T06:39:01Z,NaN
2,3a3d4c90-0b6e-45dc-b1e6-014659055bbf,A G,https://play-lh.googleusercontent.com/a-/ALV-U...,3.0,"Fun game, I enjoy the story. There is tons to ...",11,"October 30, 2024",2024-10-30T18:01:37Z,NaN
3,99015538-1d26-4bd9-a02f-37bc2a361d1a,Astra,https://play-lh.googleusercontent.com/a-/ALV-U...,5.0,This game is phenomenal. The art style and sce...,77,"October 18, 2024",2024-10-18T19:11:12Z,NaN
4,541b3b4d-97f6-42e0-9c68-059a63e1e67f,Angela Williams,https://play-lh.googleusercontent.com/a-/ALV-U...,2.0,"I love this game. However, it is incredibly la...",14,"October 14, 2024",2024-10-14T03:04:43Z,NaN
5,ad484b6a-1b9c-42ab-9cee-9df6e28f12d4,Valerie,https://play-lh.googleusercontent.com/a-/ALV-U...,3.0,I used to be obsessed with this game but I hav...,99,"October 13, 2024",2024-10-13T14:02:08Z,NaN
6,fb25cdf6-40d8-44e8-b1b9-d439ebd88565,Daniel “Chotara” Ricciardi,https://play-lh.googleusercontent.com/a-/ALV-U...,5.0,Highly recommend. I have been playing the game...,65,"October 18, 2024",2024-10-18T15:20:11Z,NaN
7,36e821d3-9441-4eaa-94a7-9c7b9b7463b5,Amy,https://play-lh.googleusercontent.com/a-/ALV-U...,4.0,Hello! I absolutely love this game! It has inc...,29,"October 29, 2024",2024-10-29T00:27:08Z,NaN
8,4b1e6dcb-d251-450e-9be6-358b4bb8e9d6,Feitan Desy,https://play-lh.googleusercontent.com/a-/ALV-U...,4.0,I've been playing this game since January 16th...,64,"October 08, 2024",2024-10-08T23:13:09Z,NaN
9,27525772-1c0a-40e4-8321-4c5f0a0f7c64,Olivia Staringer,https://play-lh.googleusercontent.com/a-/ALV-U...,1.0,"Uninteresting characters, uninteresting dialog...",14,"November 11, 2024",2024-11-11T22:58:07Z,NaN


In [3]:
import pandas as pd
import os

# Membaca data dari file CSV
df = pd.read_csv('data/google-play-rev-gen-2.csv')

# Memilih kolom yang dibutuhkan
columns_to_keep = ['rating', 'snippet', 'likes', 'date', 'iso_date', 'response']
filtered_df = df[columns_to_keep]

# Membuat direktori jika belum ada
output_dir = 'new-review-data'
os.makedirs(output_dir, exist_ok=True)

# Menyimpan data baru ke file CSV
output_file = os.path.join(output_dir, 'filtered_reviews_gensin.csv')
filtered_df.to_csv(output_file, index=False)

print(f"new Data will be saved to {output_file}")

new Data will be saved to new-review-data/filtered_reviews_gensin.csv


Inisialisasi stopwords dan stemmer

Preprocessing

In [2]:
import pandas as pd
import os
import spacy
import re

# Muat model bahasa spaCy (pastikan sudah menginstal spaCy dan model bahasa)
nlp = spacy.load("en_core_web_sm")

# Fungsi preprocessing menggunakan spaCy
def preprocess_text(text):
    # Menghapus karakter khusus dan angka
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    
    # Tokenisasi dengan spaCy
    doc = nlp(text.lower())
    
    # Lemmatization dan menghapus stopwords
    processed_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    # Gabungkan kembali tokens yang telah diproses menjadi teks yang bersih
    return ' '.join(processed_tokens)

# Baca dataset filtered_reviews.csv
input_file = 'new-review-data/filtered_reviews_gensin.csv'
df = pd.read_csv(input_file)

# Terapkan preprocessing pada kolom 'Snippet'
df['cleaned_snippet'] = df['snippet'].apply(preprocess_text)  # Pastikan nama kolom case-sensitive

# Hapus kolom 'Snippet' yang asli
df = df.drop(columns=['snippet'])

# Buat direktori baru untuk menyimpan hasil
output_dir = 'new-review-data/filter-snippet'
os.makedirs(output_dir, exist_ok=True)

# Simpan dataset yang diperbarui
output_file = os.path.join(output_dir, 'filtered_snippet_reviews_gensin.csv')
df.to_csv(output_file, index=False)

print(f"New data will be processed with cleaned text to new data {output_file}")

New data will be processed with cleaned text to new data new-review-data/filter-snippet/filtered_snippet_reviews_gensin.csv


Keyword Extraction TF-IDF

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Baca dataset baru yang telah diproses
input_file = 'new-review-data/filter-snippet/filtered_snippet_reviews_gensin.csv'
df = pd.read_csv(input_file)

# Pastikan kolom 'cleaned_snippet' ada
if 'cleaned_snippet' not in df.columns:
    raise ValueError("Kolom 'cleaned_snippet' tidak ditemukan dalam dataset. Periksa file input.")

# Inisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')  # Gunakan stop_words built-in dari scikit-learn

# Fit dan transform data ulasan yang telah diproses menggunakan TF-IDF
X_tfidf = vectorizer.fit_transform(df['cleaned_snippet'])  # Gunakan 'cleaned_snippet' yang sudah diproses

# Ambil fitur kata kunci dari hasil TF-IDF
tfidf_features = vectorizer.get_feature_names_out()

# Konversi hasil TF-IDF menjadi DataFrame agar lebih mudah dibaca
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_features)

# Tampilkan hasil TF-IDF untuk 5 baris pertama
print(tfidf_df.head())

# Simpan hasil TF-IDF ke file baru
output_file = 'new-review-data/filter-snippet/tfidf_filtered_snippet_reviews_gensin.csv'
tfidf_df.to_csv(output_file, index=False)

print(f"Hasil TF-IDF disimpan ke {output_file}")


    ability      able  absolute  absolutely  absurd  abt  abyss  acc  accept  \
0  0.000000  0.000000       0.0         0.0     0.0  0.0    0.0  0.0     0.0   
1  0.229202  0.000000       0.0         0.0     0.0  0.0    0.0  0.0     0.0   
2  0.000000  0.200019       0.0         0.0     0.0  0.0    0.0  0.0     0.0   
3  0.000000  0.000000       0.0         0.0     0.0  0.0    0.0  0.0     0.0   
4  0.000000  0.000000       0.0         0.0     0.0  0.0    0.0  0.0     0.0   

   accidentally  ...  yeah      year  yep       yes  yesterday  youtube  \
0           0.0  ...   0.0  0.000000  0.0  0.000000        0.0      0.0   
1           0.0  ...   0.0  0.000000  0.0  0.000000        0.0      0.0   
2           0.0  ...   0.0  0.143681  0.0  0.000000        0.0      0.0   
3           0.0  ...   0.0  0.000000  0.0  0.209655        0.0      0.0   
4           0.0  ...   0.0  0.000000  0.0  0.000000        0.0      0.0   

   youtuber  yrs  zelda  zero  
0       0.0  0.0    0.0   0.0  
1   

Menampilkan Kata Kunci Berdasarkan Skor TF-IDF


In [6]:
# Menampilkan kata-kata dengan skor TF-IDF tertinggi pada dokumen pertama
tfidf_scores = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_features)
top_words = tfidf_scores.iloc[0].sort_values(ascending=False).head(10)
print(f"Top 10 kata kunci untuk dokumen pertama:\n{top_words}")

Top 10 kata kunci untuk dokumen pertama:
energy       0.368646
upgrade      0.338216
consume      0.210333
response     0.210333
excessive    0.210333
hassle       0.210333
struggle     0.210333
chore        0.195118
opinion      0.195118
cap          0.195118
Name: 0, dtype: float64


In [7]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import RobertaTokenizer, RobertaModel

# Muat model RoBERTa dan tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# Fungsi untuk mendapatkan embeddings dari teks menggunakan RoBERTa
def get_roberta_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Rata-rata embeddings dari token terakhir
    return embeddings.squeeze().numpy()

# Baca dataset yang sudah diproses
input_file = 'new-review-data/filter-snippet/filtered_snippet_reviews_gensin.csv'
df = pd.read_csv(input_file)

# Ambil ulasan yang telah diproses (misal kolom 'cleaned_snippet')
if 'cleaned_snippet' not in df.columns:
    raise ValueError("Kolom 'cleaned_snippet' tidak ditemukan dalam dataset. Periksa file input.")

# Ambil embeddings untuk setiap ulasan
embeddings = [get_roberta_embeddings(text) for text in df['cleaned_snippet']]

# Hitung cosine similarity antara setiap pasang ulasan
cosine_similarities = cosine_similarity(embeddings)

# Menampilkan hasil cosine similarity
print(cosine_similarities)

# Menyimpan hasil similarity ke dalam DataFrame
cosine_sim_df = pd.DataFrame(cosine_similarities)

# Simpan hasil cosine similarity ke file baru
output_file = 'new-review-data/filter-snippet/cosine_similarity_gensin_reviews.csv'
cosine_sim_df.to_csv(output_file, index=False)

print(f"Hasil cosine similarity disimpan ke {output_file}")


/home/rebecca/Campus/System-Retrieval-Information/system-retrieval-information/gensin/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[[0.99999976 0.97253555 0.9862162  ... 0.96884036 0.9793917  0.9839121 ]
 [0.97253555 0.99999976 0.9705523  ... 0.9612145  0.96226656 0.9759674 ]
 [0.9862162  0.9705523  1.0000005  ... 0.9731432  0.9789413  0.981642  ]
 ...
 [0.96884036 0.9612145  0.9731432  ... 0.9999999  0.9755378  0.9726212 ]
 [0.9793917  0.96226656 0.9789413  ... 0.9755378  0.9999999  0.9841534 ]
 [0.9839121  0.9759674  0.981642   ... 0.9726212  0.9841534  0.9999996 ]]
Hasil cosine similarity disimpan ke new-review-data/filter-snippet/cosine_similarity_gensin_reviews.csv


In [10]:
# Fungsi untuk merekomendasikan ulasan berdasarkan kesamaan cosine
def recommend_reviews(idx, cosine_sim_matrix, top_n=10):  # Mengubah top_n menjadi 10
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))  # Dapatkan skor kesamaan untuk review tertentu
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Urutkan berdasarkan kesamaan
    sim_scores = sim_scores[1:top_n+1]  # Ambil top N ulasan yang mirip
    review_indices = [score[0] for score in sim_scores]
    return review_indices

# Contoh: Mengambil rekomendasi untuk ulasan pertama
recommended_reviews = recommend_reviews(0, cosine_similarities, top_n=10)  # Mengubah top_n menjadi 10

# Tampilkan ulasan yang direkomendasikan dengan penomoran
print("\nRekomendasi untuk review pertama:")
for i, idx in enumerate(recommended_reviews, 1):  # Menambahkan penomoran
    print(f"{i}. {df.iloc[idx]['cleaned_snippet']}")



Rekomendasi untuk review pertama:
1. ve play game year experience player go gameplay good easy understand elemental reaction easy remember accommodate storyline plot good character pretty write love detail people know experience glitch fix time overall highly recommend game fun great time pass bore
2. initially star review long review anymore good faith love art style game music amazing world beautiful month ago finish main story new patch update drop main story finish quickly character story quest game get stagnet thing ask hsr instead personally annoying love
3. hard good rating thing worth play genshin impact story story like inazuma bad music rest experience gameplay level character talent artifact exploration quality life fix worth watch youtube story listen spotify music
4. genshin deserve   star review great game nice quality character design story quest wonderful main story quest world quest annoying sit uninteresting dialogue end day perfectly fine need bit patience fit lot p

In [11]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.nn.functional import softmax

# Muat model RoBERTa dan tokenizer untuk analisis sentimen
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Fungsi untuk menganalisis sentimen dari teks
def sentiment_analysis(text):
    # Tokenisasi teks
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Prediksi sentimen
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    # Konversi logits ke probabilitas menggunakan softmax
    probs = softmax(logits, dim=-1)
    
    # Tentukan label sentimen (0 = Negatif, 1 = Positif)
    sentiment = torch.argmax(probs, dim=-1).item()
    
    # Kembalikan hasil sentimen dan probabilitasnya
    sentiment_label = 'Positif' if sentiment == 1 else 'Negatif'
    sentiment_prob = probs[0][sentiment].item()
    
    return sentiment_label, sentiment_prob

# Baca dataset yang sudah diproses
input_file = 'new-review-data/filter-snippet/filtered_snippet_reviews_gensin.csv'
df = pd.read_csv(input_file)

# Pastikan kolom 'cleaned_snippet' ada
if 'cleaned_snippet' not in df.columns:
    raise ValueError("Kolom 'cleaned_snippet' tidak ditemukan dalam dataset. Periksa file input.")

# Terapkan analisis sentimen untuk setiap ulasan
df['sentiment'], df['sentiment_prob'] = zip(*df['cleaned_snippet'].apply(sentiment_analysis))

# Simpan hasil analisis sentimen ke file baru
output_file = 'new-review-data/filter-snippet/sentiment_analysis_gensin_reviews.csv'
df.to_csv(output_file, index=False)

# Tampilkan beberapa contoh hasil sentimen
print(df[['cleaned_snippet', 'sentiment', 'sentiment_prob']].head())

print(f"Hasil analisis sentimen disimpan ke {output_file}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                                     cleaned_snippet sentiment  sentiment_prob
0  ve review decide edit response current opinion...   Negatif        0.561774
1  fun wish fighting quest bunch talk run come qu...   Negatif        0.565175
2  fun game enjoy story ton ton content clear bor...   Negatif        0.562104
3  game phenomenal art style scenery stunning yes...   Negatif        0.560393
4  love game incredibly laggy point unplayable gr...   Negatif        0.561833
Hasil analisis sentimen disimpan ke new-review-data/filter-snippet/sentiment_analysis_gensin_reviews.csv


In [12]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Baca dataset yang sudah diproses (hasil analisis sentimen)
input_file = 'new-review-data/filter-snippet/sentiment_analysis_gensin_reviews.csv'
df = pd.read_csv(input_file)

# Misalkan kita memiliki ground truth tentang relevansi ulasan (untuk keperluan evaluasi)
# Kolom 'relevance' adalah ground truth, yang bisa berisi 1 untuk relevan (positif) dan 0 untuk tidak relevan (negatif)
# Contoh data ground truth, sesuaikan dengan dataset kamu
df['relevance'] = df['sentiment'].apply(lambda x: 1 if x == 'Positif' else 0)

# Split data untuk evaluasi (bisa menggunakan validasi silang atau split data)
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_snippet'], df['relevance'], test_size=0.3, random_state=42)

# Fungsi untuk menganalisis sentimen dan mendapatkan prediksi
def get_sentiment_predictions(texts):
    predictions = []
    for text in texts:
        sentiment, _ = sentiment_analysis(text)  # Gunakan fungsi sentiment_analysis yang sudah dibuat
        predictions.append(1 if sentiment == 'Positif' else 0)  # 1 untuk positif, 0 untuk negatif
    return predictions

# Ambil prediksi sentimen untuk data uji
y_pred = get_sentiment_predictions(X_test)

# Hitung metrik evaluasi: Precision, Recall, F1-Score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Tampilkan hasil evaluasi
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000


/home/rebecca/Campus/System-Retrieval-Information/system-retrieval-information/gensin/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rebecca/Campus/System-Retrieval-Information/system-retrieval-information/gensin/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rebecca/Campus/System-Retrieval-Information/system-retrieval-information/gensin/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true n